<a href="https://colab.research.google.com/github/TatyanaLanceva/Netology/blob/main/4_3_Advanced_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по правилам:
*   если источник traffic_source равен Yandex или Google, то в source_type ставится organic;
*   для источников paid и email из России ставим ad;
*   для источников paid и email не из России ставим other;
*   все остальные варианты берём из traffic_source без изменений.

[log.csv](https://drive.google.com/open?id=16bueO9McYFCtnGiyB7aWQAKSV2NyeC2J&usp=drive_fs)


In [ ]:
import gdown
import pandas as pd
import re
from datetime import timedelta


In [ ]:
url = 'https://drive.google.com/uc?id=16bueO9McYFCtnGiyB7aWQAKSV2NyeC2J'
output = 'log.csv'
gdown.download(url, output, quiet=False)
log = pd.read_csv('log.csv', sep=';')

Downloading...
From: https://drive.google.com/uc?id=16bueO9McYFCtnGiyB7aWQAKSV2NyeC2J
To: /content/log.csv
100%|██████████| 1.55M/1.55M [00:00<00:00, 13.4MB/s]


In [ ]:
# Если источник traffic_source равен Yandex или Google, то в source_type ставится organic:

organic = (log['traffic_source'] == 'yandex') | (log['traffic_source'] == 'google')

# Для источников paid и email из России ставим ad:

paid_russia = ((log['traffic_source'] == 'paid') | (log['traffic_source'] == 'email')) & (log['region'] == 'Russia')

# Для источников paid и email не из России ставим other:

paid_other = ((log['traffic_source'] == 'paid') | (log['traffic_source'] == 'email')) & (log['region'] != 'Russia')

# Установка значений в столбце 'source_type' на основе условий
log.loc[organic, 'source_type'] = 'organic'
log.loc[paid_russia, 'source_type'] = 'ad'
log.loc[paid_other, 'source_type'] = 'other'

# Все остальные варианты берём из traffic_source без изменений

log['source_type'] = log['source_type'].fillna(log['traffic_source'])

log.head(-5)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
...,...,...,...,...,...,...,...
18928,1550094255,c7e4bce62a,https://host.ru/8880fd00cf23066541,Australia,e721d23ec6,yandex,organic
18929,1550094260,39aeec73f8,https://host.ru/ee7c4e0674a284ee3f,Russia,9c832cf9d4,direct,direct
18930,1550094269,f1b4590410,https://host.ru/a5dda93e70318570c0,Russia,7f59a1e6a7,direct,direct
18931,1550094269,a7e2dfd759,https://host.ru/ef57543002ce2a0f2a,Belarus,04fec64cda,direct,direct


In [ ]:
# Проверим, как заполнился столбец source_type, видим, что здесь есть только требуемые значения:

log.groupby('source_type').count().head()

,timestamp,visit_id,url,region,user_id,traffic_source
source_type,,,,,,
ad,3790,3790,3790,3790,3790,3790
direct,7013,7013,7013,7013,7013,7013
organic,5407,5407,5407,5407,5407,5407
other,2728,2728,2728,2728,2728,2728


# Задание 2

В файле URLs.txt содержатся URL страниц новостного сайта.

**Вам нужно отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис.**

Выполните действия:
*   Прочитайте содержимое файла с датафрейм.
*   Отфильтруйте страницы с текстом новостей, используя метод str.contains и    регулярное выражение в соответствие с заданным шаблоном.

[URLs.txt](https://drive.google.com/open?id=17-vrKJy9H-FuhdrNdiGquP9HBJedz78h&usp=drive_fs)


In [ ]:
# Загрузка данных:

url = 'https://drive.google.com/uc?id=17-vrKJy9H-FuhdrNdiGquP9HBJedz78h'
output = 'urls.txt'
gdown.download(url, output, quiet=False)
urls = pd.read_table('urls.txt', sep=';')
urls.head()


Downloading...
From: https://drive.google.com/uc?id=17-vrKJy9H-FuhdrNdiGquP9HBJedz78h
To: /content/urls.txt
100%|██████████| 5.96k/5.96k [00:00<00:00, 11.7MB/s]


,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [ ]:
# Добавляем длину строки и сортируем по длине
urls['len_str'] = urls['url'].str.len()
urls = urls.sort_values('len_str', ascending=True)

# Находим даты в формате /YYYYMMDD- и добавляем их в новый столбец
urls['news_url'] = urls['url'].str.extract(r'(\/\d{8}-)')

# Фильтруем строки с непустыми значениями в news_url
urlsf = urls.query('news_url.notnull()')

# Проверка
print(urlsf[['url']].iloc[:10])

                                                  url
45             /video/36151719-petrosyanu-grozit-sud/
59       /articles/36213214-vozmozhna-bolshaya-voyna/
16     /starlife/36174817-guzeeva-rasstalas-s-muzhem/
57    /starlife/36240935-dzhigurde-postavili-diagnoz/
47   /articles/36193368-donbass-reshil-udarit-pervym/
41  /cis/36189830-kiev-arestoval-imuschestvo-v-krymu/
54  /finances/36043911-rublyu-predrekli-dvoynoy-udar/
60  /articles/36142564-ukrainu-ureguliruyut-po-tih...
66  /europe/36046038-v-parizhskom-metro-progremel-...
19  /cis/36229699-poroshenko-obyavil-vseukrainskiy...


# Задание 3

Используйте [файл](https://drive.google.com/open?id=171mKi3TMb8cMyrIBV9zIWAhbnhm88ecl&usp=drive_fs) с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.

In [ ]:
# Скачиваем файл с данными
url = 'https://drive.google.com/uc?id=171mKi3TMb8cMyrIBV9zIWAhbnhm88ecl'
output = 'ratings.csv'
gdown.download(url, output, quiet=False)
ratings = pd.read_csv(output, sep=',')
ratings.head()

Downloading...
From: https://drive.google.com/uc?id=171mKi3TMb8cMyrIBV9zIWAhbnhm88ecl
To: /content/ratings.csv
100%|██████████| 2.44M/2.44M [00:00<00:00, 18.6MB/s]


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
# Отфильтруем пользователей, поставивших более 100 оценок:

users = ratings.groupby('userId')['movieId'].count().reset_index(name='count')
users_100 = users[users["count"] > 100]
users_top = ratings.merge(users_100, on='userId', how='inner')

# Вычислим разницу между максимумом и минимумом по столбцу timestamp для пользователей с оценками больше 100 штук:

lifetime_df = users_top.groupby('userId').agg({'timestamp': ['max', 'min']})
lifetime_df.columns = ['max_timestamp', 'min_timestamp']
lifetime_df['lifetime'] = lifetime_df['max_timestamp'] - lifetime_df['min_timestamp']

# Найдем среднее время жизни пользователей, которые выставили более 100 оценок и переведем в минуты:
average_lifetime = lifetime_df['lifetime'].mean()
average_lifetime_timedelta = timedelta(seconds=int(average_lifetime))
average_lifetime_minutes = average_lifetime_timedelta.total_seconds() / 60

print(f'Среднее время жизни пользователей, поставивших более 100 оценок: {average_lifetime_minutes:.0f} мин.')

Среднее время жизни пользователей, поставивших более 100 оценок: 668008 мин.


# Задание 4

Дана статистика услуг перевозок клиентов компании по типам.
Нужно сформировать две таблицы:

*   таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;
*   аналогичную таблицу по типам выручки с указанием адреса клиента.

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

*  rzd - железнодорожные перевозки
*  auto - автомобильные перевозки
*  air - воздушные перевозки
*  client_base - адреса клиентов


In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [ ]:
# Объединим значения без потерь строк, используя merge, заменим NaN на пустые значения:

result = (
    client_base
    .merge(air, on='client_id', how='outer')
    .merge(auto, on='client_id', how='outer')
    .merge(rzd, on='client_id', how='outer')
).fillna('')

result

,client_id,address,air_revenue,auto_revenue,rzd_revenue
0,111,Комсомольская 4,,,1093.0
1,112,Энтузиастов 8а,,,2810.0
2,113,Левобережная 1а,,57483.0,10283.0
3,114,Мира 14,,83.0,5774.0
4,115,ЗЖБИиДК 1,81.0,912.0,981.0
5,116,Строителей 18,4.0,4834.0,
6,117,Панфиловская 33,13.0,98.0,
7,118,Мастеркова 4,173.0,,


In [ ]:
#Удалим столбец с адресом и выведем на печать:

result.drop(columns=['address'], inplace=True)
result

,client_id,air_revenue,auto_revenue,rzd_revenue
0,111,,,1093.0
1,112,,,2810.0
2,113,,57483.0,10283.0
3,114,,83.0,5774.0
4,115,81.0,912.0,981.0
5,116,4.0,4834.0,
6,117,13.0,98.0,
7,118,173.0,,
